In [1]:
from dateutil.relativedelta import relativedelta
from dateutil.utils import today
from eolearn.core import (
    FeatureType,
    SaveTask,
    linearly_connect_tasks,
    EOWorkflow,
    EOExecutor,
    OutputTask,
)
from eolearn.io import get_available_timestamps, SentinelHubInputTask
from sentinelhub import SHConfig, BBox, CRS, DataCollection
import matplotlib.pyplot as plt
from eolearn.core import EOPatch
import numpy as np
import geopandas as gpd
from pathlib import Path
from importlib import reload
from matplotlib import pyplot as plt

C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [82]:
config = SHConfig(profile="sentinel-dl")

In [83]:
roi_bbox = BBox(bbox=(5.60, 52.68, 5.75, 52.63), crs=CRS.WGS84)

In [78]:
def get_last_two_timestamps(data_collection):
    today_ts = today()
    two_months_ago_ts = today_ts - relativedelta(months=2)
    time_of_interest = (two_months_ago_ts, today_ts)

    timestamps = get_available_timestamps(
        bbox=roi_bbox,
        time_interval=time_of_interest,
        data_collection=data_collection,
        config=config,
    )
    return timestamps[-2:]

In [5]:
# max cloud cover
maxcc = 0.8

# meters
resolution = 20

data_collection = DataCollection.SENTINEL2_L1C

time_interval = get_last_two_timestamps(data_collection)

input_task = SentinelHubInputTask(
    data_collection=data_collection,
    # bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"] # if not passed, it will download all
    bands_feature=(FeatureType.DATA, "L1C_data"),
    additional_data=[(FeatureType.MASK, "dataMask")],  # cloud mask
    maxcc=maxcc,
    resolution=resolution,
    config=config,  # important since we are using sentinel-dl, alternatively save ID and secret to default profile
)

besides L1C we can also get L2A, SCL, NDWI... Refer to this guide: https://github.com/sentinel-hub/eo-learn/blob/master/examples/io/SentinelHubIO.ipynb

In [6]:
save = SaveTask("eopatch_fd")
output_task = OutputTask("eopatch")

In [7]:
workflow_nodes = linearly_connect_tasks(input_task, save, output_task)
workflow = EOWorkflow(workflow_nodes)
# workflow.dependency_graph() additionally install graphviz and eo-learn[VISUALIZATION]

In [8]:
# take nodes, these are not the same as tasks!!!
input_node = workflow_nodes[0]
save_node = workflow_nodes[-2]

execution_args = [
    {
        input_node: {"bbox": roi_bbox, "time_interval": time_interval},
        save_node: {"eopatch_folder": "eopatch_file"},
    }
]

executor = EOExecutor(workflow, execution_args, logs_folder="./reports")

In [10]:
res = executor.run(workers=4)

executor.make_report()

failed_ids = executor.get_failed_executions()
if failed_ids:
    raise RuntimeError(
        f"Execution failed EOPatches with IDs:\n{failed_ids}\n"
        f"For more info check report at {executor.get_report_path()}"
    )

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [11]:
eopatch = res[0].outputs["eopatch"]
eopatch

EOPatch(
  bbox=BBox(((5.6, 52.63), (5.75, 52.68)), crs=CRS('4326'))
  timestamps=[datetime.datetime(2024, 10, 26, 10, 56, 23)]
  mask={
    dataMask: numpy.ndarray(shape=(1, 297, 497, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 297, 497, 13), dtype=float32)
  }
)

In [20]:
eopatch.plot((FeatureType.DATA, "L1C_data"), times=[0], rgb=[3, 2, 1])
plt.show()

In [69]:
import util.region as rt

reload(rt)
slo = rt.prepare_slo_shape()

country_shape = slo.geometry.values[0]

plt.axis("off")
country_width = country_shape.bounds[2] - country_shape.bounds[0]
country_height = country_shape.bounds[3] - country_shape.bounds[1]
print(f"Dimension of the area is {country_width:.0f} x {country_height:.0f} m2")
slo.plot()
plt.show()

Dimension of the area is 243208 x 161608 m2


C:\Users\blazr\AppData\Local\Temp\ipykernel_10980\1485639266.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [70]:
import util.region as rt

reload(rt)
chunks, bbox_list = rt.prepare_slo_chunks()
chunks

,index,index_x,index_y,geometry
0,0,0,19,"POLYGON ((368640.000 5125120.000, 368640.000 5..."
1,1,1,17,"POLYGON ((373760.000 5114880.000, 373760.000 5..."
2,2,1,18,"POLYGON ((373760.000 5120000.000, 373760.000 5..."
3,3,1,19,"POLYGON ((373760.000 5125120.000, 373760.000 5..."
4,4,1,20,"POLYGON ((373760.000 5130240.000, 373760.000 5..."
...,...,...,...,...
895,895,47,25,"POLYGON ((609280.000 5155840.000, 609280.000 5..."
896,896,47,26,"POLYGON ((609280.000 5160960.000, 609280.000 5..."
897,897,48,23,"POLYGON ((614400.000 5145600.000, 614400.000 5..."
898,898,48,24,"POLYGON ((614400.000 5150720.000, 614400.000 5..."


In [25]:
chunks.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [73]:
fig, ax = plt.subplots(figsize=(30, 30))
chunks, bbox_list = rt.prepare_slo_chunks(
    resolution=20, patch_size=512, fixed_meter_patch_size=5120
)
ax.set_title("Tiles of Slovenia", fontsize=25)
slo.plot(ax=ax, facecolor="w", edgecolor="b", alpha=0.5)
chunks.plot(ax=ax, facecolor="w", edgecolor="r", alpha=0.5)

for i, row in chunks.iterrows():
    geo = row["geometry"]
    ax.text(geo.centroid.x, geo.centroid.y, row["index"], ha="center", va="center")

plt.axis("off")
plt.show()
plt.savefig("box.png")

C:\Users\blazr\AppData\Local\Temp\ipykernel_10980\777527452.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(30, 30))


Fixing bbox chunk size to 5120, resolution and patch_size are ignored.


C:\Users\blazr\AppData\Local\Temp\ipykernel_10980\777527452.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [2]:
resolution = 10
patch_size = 512
collection = DataCollection.SENTINEL2_L1C
config = SHConfig("sentinel-dl")
start_date = "29-10-2024"
maxcc = 0.2

import util.region as rt

reload(rt)
import util.workflows as wf

reload(wf)
import util.time as t

reload(t)

from util.workflows import prepare_workflow, execute_flow
from util.region import prepare_slo_chunks
from util.time import get_last_month_span

collection = DataCollection.SENTINEL2_L1C
_, bbox_list = prepare_slo_chunks(resolution=resolution, patch_size=patch_size)
workflow, node_map = prepare_workflow(
    out_dir="./patches",
    config=config,
    resolution=resolution,
    data_collection=collection,
    maxcc=maxcc,
)

time_of_interest = get_last_month_span(start_date)

execute_flow(
    workflow=workflow,
    node_map=node_map,
    bbox_list=bbox_list,
    num_workers=4,
    time_interval=time_of_interest,
)

Setting chunk sizes to 5120x5120 m^2.


100%|██████████| 16/16 [00:24<00:00,  1.50s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [99]:
ld = EOPatch.load("patches/patch_0", lazy_loading=True)
ld.data["data"].shape

(1, 512, 512, 13)

In [4]:
# Draw the RGB images
fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))

for i in range(15):
    eopatch_path = Path("./patches") / start_date / f"patch_{i}"
    eopatch = EOPatch.load(eopatch_path, lazy_loading=True)

    ax = axs[i // 4][i % 4]
    ax.imshow(np.clip(eopatch.data["data"][0][..., [3, 2, 1]] * 3.5, 0, 1))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    del eopatch

fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig("geo_l.png")

In [88]:
# Draw the RGB images
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(20, 20))

for p_idx, (i, j) in enumerate(zip([0, 2, 1, 0], [0, 1, 1, 1])):
    eopatch_path = Path("./patches") / start_date / f"patch_{p_idx}"
    eopatch = EOPatch.load(eopatch_path, lazy_loading=True)

    ax = axs[i][j]
    ax.imshow(np.clip(eopatch.data["data"][0][..., [3, 2, 1]], 0, 1))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    del eopatch

fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig("geo.png")

In [33]:
# Draw the RGB images resolution=20
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(20, 20))

for p_idx, (i, j) in enumerate(zip([0], [0])):
    eopatch_path = Path("./patches2") / f"patch_{p_idx}"
    eopatch = EOPatch.load(eopatch_path, lazy_loading=True)

    ax = axs[i][j]
    ax.imshow(np.clip(eopatch.data["data"][0][..., [3, 2, 1]], 0, 1))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    del eopatch

fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig("geo2.png")